# Creating the Dataframe and Collecting the Average Green Values

This notebook takes images in the prepared images directory (created in notebook 0.2) to create a dataframe called "Oat_Data" and save it as a csv. It also collects the average green value from each image. First, an archive folder will be created if it does not already exist. Next, if "Oat_Data.csv" already exists, it will move it to the archive directory and append the date to the file name. Finally, It will create  a directory that has plot, date, crop variety, cover crop method, and average green value.

-Luke Rosen
4/25/2022

Update 5/16/2022

Have Green Average be normalized.

Update 5/19/2022

Changed "Mean_Green" to "Green_Avg".

In [1]:
import os
import numpy as np
import glob
import skimage
from skimage.io import imread, imsave, imshow
import pandas as pd
from datetime import datetime

In [2]:
#Creates Archive folder if it doesn't exist
if not os.path.exists("./Archive"):
    os.mkdir("./Archive") 
Old = "./Archive/"

In [3]:
#Moves csv if exists (Oat_Data_m_d.csv)
if os.path.exists("Oat_Data.csv"):
    os.rename("Oat_Data.csv", Old + "Oat_Data_" + str(datetime.today().month) + "_" + str(datetime.today().day) + ".csv")

In [4]:
img_path = "./images_prepared" #Path to prepared images

In [5]:
count = 0
Image = []           #Plot_yyyymmdd
Plot= []             #Plot
Date_Index = []      #yyyymmdd
Date = []            #dd/mm/yyyy
Variety = []         #Oat or Soy
Cover = []           #Fallow, Post, or Under 
Green_avg = []       # Average Green



for tiff in glob.glob(img_path + "\*.tif"):
    end_path = tiff.split("\\")[-1] #Getting Image Name (Plot_yyyymmdd.tif)
    name = end_path.split(".")[0]       # Remove 'tif'   (Plot_yyyymmdd)
    Image.append(name)
    plot, date = name.split("_")  #[plot, yyyymmdd]
    Plot.append(plot)
    Date_Index.append(date)
    new_date = datetime.strptime(str(date), "%Y%m%d").strftime("%m/%d/%Y") # yyyymmdd to dd/mm/yyyy
    Date.append(new_date)
    if plot.startswith("106") or plot.startswith("203") or plot.startswith("303") or plot.startswith("404"): ##Conditions of variety
        crop = "Soy"
        Variety.append(crop)
    else:
        crop = "Oat"
        Variety.append(crop)
    if plot.startswith("1") and plot.endswith("A") or not plot.startswith("1") and plot.endswith("B"): ##Conditions of cover
        cover = "Fallow"
        Cover.append(cover)
    elif plot.startswith("1") and plot.endswith("C") or plot.startswith("3") and plot.endswith("C") or plot.startswith("2") and plot.endswith("A") or plot.startswith("4") and plot.endswith("A"):
        cover = "Under"
        Cover.append(cover)
    else:
        cover = "Post"
        Cover.append(cover)
    with np.errstate(divide = "ignore", invalid = "ignore"):
        img = imread(tiff)
        
        Red_c = img[:,:,0] * 1.0     ## Red color band as float

        Green_c = img[:,:,1] * 1.0   ## Green color band as float
 
        Blue_c = img[:,:,2] * 1.0    ## Blue color band as fload
        
        Green_n = Green_c/(Red_c + Green_c + Blue_c) ## Normalize Green
        Green_n[np.isnan(Green_n)] = 0
        Green_n[np.isinf(Green_n)] = 0
    
        green_ave = Green_n.mean()                    ##Average green_normalized per image.  
        Green_avg.append(green_ave)
    
    if count % 42 == 0:
        print("Done with image " + name + "...")
    count=count+1
print("Creating Dataframe...")
Oat_data = pd.DataFrame({"Image":Image,         ##Creating dataframe
                        "Plot":Plot,
                        "Date_Index":Date_Index,
                        "Date":Date,
                        "Variety":Variety,
                        "Cover":Cover,
                        "Green_Avg_Image":Green_avg})        

print("Dataframe Complete!!!")

Done with image 101A_20211025...
Done with image 308C_20211025...
Done with image 205A_20211025...
Done with image 103A_20210708...
Done with image 207A_20210708...
Done with image 401A_20210708...
Done with image 103A_20210821...
Done with image 310C_20210821...
Done with image 207A_20210821...
Done with image 304C_20211015...
Done with image 201A_20211015...
Done with image 408C_20211015...
Done with image 105A_20210909...
Done with image 402C_20210909...
Done with image 209A_20210909...
Done with image 306C_20210929...
Done with image 203A_20210929...
Done with image 410C_20210929...
Done with image 107A_20211005...
Done with image 404C_20211005...
Creating Dataframe...
Dataframe Complete!!!


In [6]:
Oat_data = Oat_data.sort_values(by= ["Date", "Plot"]) ##Order by date then plot

In [7]:
Oat_data.to_csv("Oat_Data.csv", index = False)        ##Saving to csv. 

This concludes the notebook. The next notebook will use to dataframe and prepared images to create vegetation indicies. 